# RAG CODE

In [1]:
!nvidia-smi

Mon Nov 17 05:41:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip uninstall -y chromadb langchain langchain-groq langchain-community langchain-chroma langchain-text-splitters transformers sentence-transformers unstructured unstructured[pdf] langchain-huggingface
!pip install -U chromadb langchain==0.1.18 langchain-groq==0.1.5 langchain-community==0.0.18 langchain-chroma==0.1.5 langchain-text-splitters==0.0.1 langchain-huggingface==0.0.3 sentence-transformers unstructured unstructured[pdf]

In [3]:
#!apt-get install poppler-utils

In [4]:
#!pip install --upgrade unstructured[local-inference]

In [5]:
!pip install langchain
!pip install langchain-community
!pip install langchain-text-splitters
!pip install langchain-chroma

In [6]:
!pip install sentence-transformers

In [7]:
!pip install langchain-groq

In [8]:
!pip install unstructured
!pip install unstructured[pdf]

In [9]:
import os
os.environ["GROQ_API_KEY"] = ""

In [10]:
#fetch the pdf from url
import requests
url = "https://courses.cs.umbc.edu/671/fall12/notes/python/04_python_functions.pdf"

response = requests.get(url)

In [11]:
response

<Response [200]>

In [12]:
#save the pdf local file

with open("python_inbuildfunctions.pdf", "wb") as f:
   f.write(response.content)

In [13]:
from langchain_community.document_loaders import UnstructuredFileIOLoader

with open("python_inbuildfunctions.pdf", "rb") as f:
    loader = UnstructuredFileIOLoader(file=f)
    documents = loader.load()

documents  # This will be a list of Document objects

/tmp/ipython-input-1778148066.py:4: LangChainDeprecationWarning: The class `UnstructuredFileIOLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the `langchain-unstructured package and should be used instead. To use it run `pip install -U `langchain-unstructured` and import as `from `langchain_unstructured import UnstructuredLoader``.
  loader = UnstructuredFileIOLoader(file=f)


[Document(metadata={}, page_content='Functions in Python\n\nDefining Functions\n\nFunction definition begins with “def.” Function name and its arguments.\n\ndef get_final_answer(filename): “““Documentation String”””\n\nline1 line2 return total_counter\n\nThe indentation matters… First line with less indentation is considered to be outside of the function definition.\n\nThe keyword ‘return’ indicates the value to be sent back to the caller.\n\nNo header file or declaration of types of function or arguments\n\nColon.\n\nPython and Types\n\nDynamic typing: Python determines the data types of variable bindings in a program automatically\n\nStrong typing: But Python’s not casual about types, it enforces the types of objects\n\nFor example, you can’t just append an integer to a string, but must first convert it to a string\n\nx = “the answer is ” # x bound to a string y = 23 # y bound to an integer. print x + y # Python will complain!\n\nCalling a Function\n\nThe syntax for a function call i

In [14]:
!pip install langchain-text-splitters

In [15]:
from langchain_text_splitters import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size = 100, chunk_overlap = 50)

In [16]:
text_splitter

In [17]:
texts = text_splitter.split_documents(documents)

In [18]:
type(texts)

list

In [19]:
texts[10]

Document(metadata={}, page_content='Calling a Function')

In [20]:
#pip install langchain-huggingface

In [21]:
!pip install langchain-huggingface

In [22]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
persist_directory = "vector_db"

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [23]:
from langchain_chroma import Chroma

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [24]:
print(vectordb)

In [25]:
# Retrival

retriever = vectordb.as_retriever()

In [27]:
from langchain_groq import ChatGroq
llm = ChatGroq(model = "llama-3.3-70b-versatile",temperature=0.2)

In [33]:
!pip install -U langchain langchain-community langchain-groq chromadb

In [38]:
from langchain_classic.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [39]:
query = 'explain about filter, map, reduce'
qa_result = qa_chain.invoke({"query":query})

In [40]:
print(qa_result)

{'query': 'explain about filter, map, reduce', 'result': "`filter`, `map`, and `reduce` are three fundamental functions in functional programming, commonly used in languages like Python. Here's a brief overview of each:\n\n1. **Filter**:\n\t* Purpose: To select a subset of elements from a collection (e.g., list, tuple) based on a condition.\n\t* Syntax: `filter(function, iterable)`\n\t* Function: Takes a function and an iterable as input. Applies the function to each element in the iterable and returns a new iterable containing only the elements for which the function returns `True`.\n\t* Example: `filter(odd, [1, 2, 3, 4])` would return `[1, 3]` if `odd` is a function that checks if a number is odd.\n2. **Map**:\n\t* Purpose: To apply a transformation to each element in a collection.\n\t* Syntax: `map(function, iterable)`\n\t* Function: Takes a function and an iterable as input. Applies the function to each element in the iterable and returns a new iterable containing the transformed 

In [41]:
print(qa_result['result'])

`filter`, `map`, and `reduce` are three fundamental functions in functional programming, commonly used in languages like Python. Here's a brief overview of each:

1. **Filter**:
	* Purpose: To select a subset of elements from a collection (e.g., list, tuple) based on a condition.
	* Syntax: `filter(function, iterable)`
	* Function: Takes a function and an iterable as input. Applies the function to each element in the iterable and returns a new iterable containing only the elements for which the function returns `True`.
	* Example: `filter(odd, [1, 2, 3, 4])` would return `[1, 3]` if `odd` is a function that checks if a number is odd.
2. **Map**:
	* Purpose: To apply a transformation to each element in a collection.
	* Syntax: `map(function, iterable)`
	* Function: Takes a function and an iterable as input. Applies the function to each element in the iterable and returns a new iterable containing the transformed elements.
	* Example: `map(square, [1, 2, 3, 4])` would return `[1, 4, 9, 1